# Import 

In [4]:
%matplotlib qt
%gui qt

import re
import os
import sys

import numpy as np
from loguru import logger

import matplotlib.pyplot as plt 


import pyqtgraph as pg
import pyqtgraph.opengl as gl

from collections import defaultdict
from datetime import datetime

import torch
import gpytorch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import scipy.interpolate as interp

In [5]:
def gatherCSV(rootPath, outsuffix = 'Tracking'):
    '''==================================================
        Collect all EIS.csv files in the rootPath
        Parameter: 
            rootPath: current search path
            outsuffix: Saving path of EIS.csv files
        Returen:
            EISDict: a 2D-dict of EIS data
            Storage Frame: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        ==================================================
    '''
    _filename       = None
    _filepath       = None
    _trackpath      = None
    _csvpath        = None
    _sessionIndex   = None
    _channelIndex   = None
    _processed      = None

    EISDict = defaultdict(dict)

    ## Iterate session
    session_pattern = re.compile(r"(.+?)_(\d{8})_01")
    bank_pattern    = re.compile(r"([1-4])")
    file_pattern    = re.compile(r"EIS_ch(\d{3})\.csv")

    ## RootDir
    for i in os.listdir(rootPath):
        match_session = session_pattern.match(i)
        ## SessionDir
        if match_session:
            logger.info(f"Session Begin: {i}")
            _sessionIndex = match_session[2]
            for j in os.listdir(f"{rootPath}/{i}"):
                match_bank = bank_pattern.match(j)
                ## BankDir
                if match_bank:
                    logger.info(f"Bank Begin: {j}")
                    _trackpath = f"{rootPath}/{i}/{j}/{outsuffix}"
                    if not os.path.exists(_trackpath):
                        continue

                    for k in os.listdir(f"{rootPath}/{i}/{j}/{outsuffix}"):
                        match_file = file_pattern.match(k)
                        ## File
                        if match_file:
                            _filename = k
                            _filepath = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            _channelIndex = (int(match_bank[1])-1)*32+int(match_file[1])
                            
                            EISDict[_sessionIndex][_channelIndex] = f"{rootPath}/{i}/{j}/{outsuffix}/{k}"
                            
    return EISDict

In [6]:
# Data Readout
def readChannel(chID, fileDict):
    '''==================================================
        Read EIS.csv file by Channel
        Parameter: 
            chID: channel index
            fileDict: EISDict[_sessionIndex][_channelIndex] = "_filepath"
        Returen:
            freq: frequency
            Zreal: real part of impedance
            Zimag: imaginary part of impedance
        ==================================================
    '''
    chData = []
    for ssID in fileDict.keys():
        _data   = np.loadtxt(fileDict[ssID][chID], delimiter=',')
        _freq   = _data[:,0]
        _Zreal  = _data[:,1] * np.cos(np.deg2rad(_data[:,2])) 
        _Zimag  = _data[:,1] * np.sin(np.deg2rad(_data[:,2])) 
        chData.append(np.stack((_freq, _Zreal, _Zimag),axis=0))

    return np.stack(chData, axis=0)

## Data Loader

In [7]:
# rootPath = "D:/Baihm/EISNN/Dataset/01037160_归档"
# ch_id = 20  # Normal to Short, Same to GPR  
# ch_id = 89  # Same to GPR  
# ch_id = 7  # Normal Example

# rootPath = "D:/Baihm/EISNN/Dataset/05087163_归档"
# ch_id = 7   # one outlier
# ch_id = 50  # No outlier but in two Phases
# ch_id = 55  # One outlier &wired end point
# ch_id = 114 # Open Circuit with on outpler

# rootPath = "D:/Baihm/EISNN/Archive/02067447_归档"
# ch_id = 68  # Short all the time

# rootPath = "D:/Baihm/EISNN/Archive/01067095_归档"
# ch_id = 19    # First Sample is outlier

# rootPath = "D:/Baihm/EISNN/Archive/09290511_归档"
# ch_id = 13    # Up & Down, 2 outliers
# ch_id = 21    # Normal + 2 outlier
# ch_id = 41    # Normal + 2 outlier - *(Hard To Tell)
# ch_id = 79    # 3-class, What a mess

# rootPath = "D:/Baihm/EISNN/Archive/11057712_归档"
# ch_id = 106    # Very Good Electrode with 1 hidden outlier, and one phase shift

# rootPath = "D:\Baihm\EISNN\Archive/10057084_归档"
# ch_id = 16    # Totaly Mess
# ch_id = 18    # Totaly Mess

rootPath = "D:\Baihm\EISNN\Archive/11067223_归档"
ch_id = 124     # Perfect with one outlier

# rootPath = "D:\Baihm\EISNN\Archive/15361101_归档"
# ch_id = 0     # Only One Sample - Run With Error


# rootPath = "D:\Baihm\EISNN\Archive/11207147_归档"
# ch_id = 0     # Only Three Sample - Run With Error

# freq_list = np.linspace(0,np.shape(chData)[2]-1,101,dtype=int)
freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
EISDict = gatherCSV(rootPath)
chData = readChannel(ch_id, EISDict)[:,:,freq_list]

2025-04-04 18:49:15.487 | INFO     | __main__:gatherCSV:32 - Session Begin: 11067223_20241230_01
2025-04-04 18:49:15.488 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:49:15.490 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:49:15.491 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:49:15.492 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-04 18:49:15.493 | INFO     | __main__:gatherCSV:32 - Session Begin: 11067223_20250101_01
2025-04-04 18:49:15.494 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:49:15.495 | INFO     | __main__:gatherCSV:38 - Bank Begin: 2
2025-04-04 18:49:15.496 | INFO     | __main__:gatherCSV:38 - Bank Begin: 3
2025-04-04 18:49:15.497 | INFO     | __main__:gatherCSV:38 - Bank Begin: 4
2025-04-04 18:49:15.498 | INFO     | __main__:gatherCSV:32 - Session Begin: 11067223_20250102_01
2025-04-04 18:49:15.499 | INFO     | __main__:gatherCSV:38 - Bank Begin: 1
2025-04-04 18:49:15.500 | INFO    

## Data Plot

In [8]:
if True:
    fig, axis = plt.subplots(1,4,figsize=(15,6))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(np.shape(chData)[0]):
    # for i in [0,4,2,11]:
        ch_eis = chData[i,:,:]
        # ch_eis = EIS_recal(chData[i,:,:])[:,freq_list]
        _color = cmap(i/np.shape(chData)[0])
        axis[0].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2, label=f"Session {i}")
        axis[1].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2, label=f"Session {i}")
        axis[2].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
        # axis[4].loglog(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2, label=f"Session {i}")
    
        # _poi_Z = np.log(np.abs(ch_eis[1,:]+1j*ch_eis[2,:]))
        # _poi_P = np.angle(ch_eis[1,:]+1j*ch_eis[2,:])
        # _poi_eis = _poi_Z * np.exp(1j*_poi_P)
        # axis[3].plot(np.real(_poi_eis), -np.imag(_poi_eis), color = _color, linewidth=2, label=f"Session {i}")
        _poi_Z = np.log(ch_eis[1,:]+1j*ch_eis[2,:])
        axis[3].plot(np.real(_poi_Z), -np.imag(_poi_Z), color = _color, linewidth=2, label=f"Session {i}")
        

# axis[0].legend(frameon=False, loc='upper left')

## Data Cleaning

In [ ]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from  Outlier import OutlierDetection

CLEAN_FLAG = True
if CLEAN_FLAG:
    eis_seq, eis_cluster, eis_anomaly, leaf_anomaly = OutlierDetection.OutlierDetection(chData)
else: 
    eis_seq = np.arange(np.shape(chData)[0])

# GPR

## EIS-GP Model

In [ ]:
# Single Point  Gaussian Process Regression
# 这个脚本中，我们把不同频率视为相互独立的变量进行考察
# 由于阻抗仍是一个复数，所以这里使用实部虚部两个task进行GP

class EISGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, num_tasks):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_tasks
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            # gpytorch.kernels.RBFKernel(),
            # gpytorch.kernels.RQKernel()
            gpytorch.kernels.MaternKernel(nu=0.5), 
            num_tasks=num_tasks, 
            rank=2
        )
        # self.covar_module.data_covar_module.lengthscale = 1
        # self.covar_module.data_covar_module.alpha = 0.001

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


def EISGPTrain(x_train, y_train, x_eval, device, training_iter = 50, lr = 0.1, OPT_TYPE = "Adam"):
    num_tasks = y_train.shape[1]
    # Initialize likelihood and model
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_tasks,
                    rank=2).to(device)
    model = EISGPModel(x_train, y_train, likelihood, num_tasks=num_tasks).to(device)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Optimizer
    if OPT_TYPE == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters
        # optimizer = torch.optim.Adam([
        #     {'params': model.covar_module.parameters(), 'lr': 0.05},  # 低学习率
        #     {'params': model.likelihood.parameters(), 'lr': 0.01},  # 适中学习率
        #     {'params': model.mean_module.parameters(), 'lr': 0.05}  # 适中学习率
        # ], lr=0.05) 




    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # logger.info(f"Training for {training_iter} iterations...")
    loss_inst       = []
    length_inst     = []
    noise_inst      = []
    if OPT_TYPE == "Adam":
        for i in range(training_iter):
            optimizer.zero_grad()
            output = model(x_train)
            loss = -mll(output, y_train)
            loss.backward()
            optimizer.step()
            
            poi_noise   = model.likelihood.noise.item()
            poi_length  = model.covar_module.data_covar_module.lengthscale.item()
            loss_inst.append(loss.item())
            length_inst.append(poi_length)
            noise_inst.append(poi_noise)
            logger.info(f"Iter {i+1}/{training_iter}\tLoss: {loss.item()}")
    
    # logger.info("Model Training Finished.")

    # Make predictions
    model.eval()
    likelihood.eval()
    with torch.no_grad(), gpytorch.settings.cholesky_jitter(1e-4):
        pred = likelihood(model(x_eval))
    # logger.info("Model Evaluation Finished.")

    return [pred, np.array(loss_inst), np.array(length_inst), np.array(noise_inst)]


## Main

In [45]:
def GPDataLoader(chData, eis_seq, eis_cluster, SPEED_RATE = 1, LOG_FLAG = True, NORM_X_FLAG = True):
    # Speed Rate = 10 means 1 day = 10 points
    x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    x_day = [x_day[i] for i in eis_seq]

    x_train = np.array([(poi - x_day[0]).days for poi in x_day])
    x_eval = np.linspace(0,max(x_train),max(x_train)*SPEED_RATE+1)

    if LOG_FLAG:
        chData_log = np.log(chData[eis_seq,1,:] + 1j*chData[eis_seq,2,:])
        y_train = np.hstack([chData_log.real,chData_log.imag])
    else: 
        y_train = np.hstack([chData[eis_seq,1,:],chData[eis_seq,2,:]])

    if NORM_X_FLAG:
        x_eval = (x_eval-x_train.mean()) / x_train.std()
        x_train = (x_train-x_train.mean()) / x_train.std()

    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)}")

    return x_train, y_train, x_eval


### Main Run

In [46]:
if True:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    x_train, y_train, x_eval = GPDataLoader(chData, eis_seq, eis_cluster, SPEED_RATE=1, LOG_FLAG=True, NORM_X_FLAG=True)
    
    x_train_tensor = torch.from_numpy(x_train).float().to(device)
    x_eval_tensor = torch.from_numpy(x_eval).float().to(device)
    y_train_tensor = torch.from_numpy(y_train).float().to(device)
    
    y_eval_tensor, loss_inst, length_inst, noise_inst = \
        EISGPTrain(x_train_tensor, y_train_tensor, x_eval_tensor, device, training_iter=500, OPT_TYPE='Adam')


    y_eval_mean = y_eval_tensor.mean.cpu().numpy()
    y_eval_cov = y_eval_tensor.covariance_matrix.cpu().detach().numpy()
    y_eval_var = y_eval_tensor.variance.detach().cpu().numpy()


2025-04-04 18:53:55.113 | INFO     | __main__:GPDataLoader:19 - 
x: (13,) 
y: (13, 202) 
x_pred(16,)


2025-04-04 18:53:55.277 | INFO     | __main__:EISGPTrain:70 - Iter 1/500	Loss: 6.318297863006592
2025-04-04 18:53:55.361 | INFO     | __main__:EISGPTrain:70 - Iter 2/500	Loss: 3.126218795776367
2025-04-04 18:53:55.455 | INFO     | __main__:EISGPTrain:70 - Iter 3/500	Loss: 2.4838414192199707
2025-04-04 18:53:55.505 | INFO     | __main__:EISGPTrain:70 - Iter 4/500	Loss: 2.4267828464508057
2025-04-04 18:53:55.549 | INFO     | __main__:EISGPTrain:70 - Iter 5/500	Loss: 2.3308494091033936
2025-04-04 18:53:55.592 | INFO     | __main__:EISGPTrain:70 - Iter 6/500	Loss: 2.252490520477295
2025-04-04 18:53:55.633 | INFO     | __main__:EISGPTrain:70 - Iter 7/500	Loss: 2.2300727367401123
2025-04-04 18:53:55.675 | INFO     | __main__:EISGPTrain:70 - Iter 8/500	Loss: 2.2268035411834717
2025-04-04 18:53:55.716 | INFO     | __main__:EISGPTrain:70 - Iter 9/500	Loss: 2.2164688110351562
2025-04-04 18:53:55.759 | INFO     | __main__:EISGPTrain:70 - Iter 10/500	Loss: 2.1978893280029297
2025-04-04 18:53:55.79

### Loss Plot

In [47]:
if True:
    fig = plt.figure()
    ax1 = fig.add_subplot(311)
    ax2 = fig.add_subplot(312)
    ax3 = fig.add_subplot(313)
    ax1.plot(loss_inst)
    ax1.set_title("Loss")
    ax2.plot(noise_inst)
    ax2.set_yscale('log')
    ax2.set_title("Noise")
    ax3.plot(length_inst)
    ax3.set_yscale('linear')
    ax3.set_title("LengthScale")
    fig.set_tight_layout(True)

### GPR Result Plot

In [48]:
if True:
    n_freq = np.shape(y_eval_mean)[1]//2
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('viridis_r')
    axis0 = fig.add_subplot(2,2,1)
    axis1 = fig.add_subplot(2,2,2)
    axis2 = fig.add_subplot(2,2,3)
    axis3 = fig.add_subplot(2,2,4)


    for i in range(n_freq):
    # for i in [50]:
        axis0.fill_between(x_eval, y_eval_mean[:,i] - 2*np.sqrt(y_eval_var[:,i]), y_eval_mean[:,i] + 2*np.sqrt(y_eval_var[:,i]), 
                        alpha=0.2, color = cmap(i/n_freq))
        
        axis1.plot(x_eval, y_eval_mean[:,i], color = cmap(i/n_freq))
        axis1.plot(x_train, y_train[:,i], 'r.')

        j = i + n_freq
        axis2.fill_between(x_eval, y_eval_mean[:,j] - 2*np.sqrt(y_eval_var[:,j]), y_eval_mean[:,j] + 2*np.sqrt(y_eval_var[:,j]), 
                        alpha=0.2, color = cmap(i/n_freq))
        
        axis3.plot(x_eval, y_eval_mean[:,j], color = cmap(i/n_freq))
        axis3.plot(x_train, y_train[:,j], 'r.')

    axis1.sharex(axis0)
    axis1.sharey(axis0)
    axis3.sharex(axis2)
    axis3.sharey(axis2)
    axis0.set_xlabel('x')
    axis0.set_ylabel('y')



### GPR EIS Plot

In [49]:
if True:
    n_freq = np.shape(y_eval_mean)[1]//2
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    axis0 = fig.add_subplot(2,3,1)
    axis1 = fig.add_subplot(2,3,2)
    axis2 = fig.add_subplot(2,3,3, projection = '3d')
    axis3 = fig.add_subplot(2,3,4)
    axis4 = fig.add_subplot(2,3,5)
    axis5 = fig.add_subplot(2,3,6, projection = '3d')


    for i in range(np.shape(x_eval)[0]):
        axis1.semilogx(chData[0,0,:], y_eval_mean[i,:n_freq], color = cmap(i/np.shape(x_eval)[0]))
        axis4.semilogx(chData[0,0,:], y_eval_mean[i,n_freq:] * 180 / np.pi, color = cmap(i/np.shape(x_eval)[0]))
        
        axis0.fill_between(chData[0,0,:], y_eval_mean[i,:n_freq] - 2*np.sqrt(y_eval_var[i,:n_freq]), y_eval_mean[i,:n_freq] + 2*np.sqrt(y_eval_var[i,:n_freq]), 
                        alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
        axis3.fill_between(chData[0,0,:], y_eval_mean[i,n_freq:] * 180 / np.pi - 2*np.sqrt(y_eval_var[i,n_freq:]) * 180 / np.pi, y_eval_mean[i,n_freq:] * 180 / np.pi + 2*np.sqrt(y_eval_var[i,n_freq:]) * 180 / np.pi, 
                        alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
    
    for i in range(np.shape(x_train)[0]):
        axis1.semilogx(chData[0,0,:], y_train[i,:n_freq], 'black', alpha = 0.3)
        axis4.semilogx(chData[0,0,:], y_train[i,n_freq:] * 180 / np.pi, 'black', alpha = 0.3)
    
    axis0.set_xscale('log')
    axis3.set_xscale('log')
    axis1.sharex(axis0)
    axis1.sharey(axis0)
    axis4.sharex(axis3)
    axis4.sharey(axis3)

    init_elev = 21  # 仰角
    init_azim = 55  # 方位角
    axis2.view_init(elev=init_elev, azim=init_azim)
    axis5.view_init(elev=init_elev, azim=init_azim)

    _x = np.arange(np.shape(x_eval)[0])
    _y = np.log10(chData[0,0,:]).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis2.plot_surface(X, Y, y_eval_mean[:,:n_freq], cmap='viridis_r', alpha=0.8)
    axis5.plot_surface(X, Y, y_eval_mean[:,n_freq:] * 180 / np.pi, cmap='viridis_r', alpha=0.8)




## Restore EIS

In [37]:
def GPRecoverEIS(y_eval_mean, y_eval_var):
    n_freq = np.shape(y_eval_mean)[1]//2
    _y_real_mean    = y_eval_mean[:,:n_freq]
    _y_imag_mean    = y_eval_mean[:,n_freq:]
    _y_real_var     = y_eval_var[:,:n_freq]
    _y_imag_var     = y_eval_var[:,n_freq:]

    # Amp
    amp_eval_mean   = np.exp(_y_real_mean + 0.5*_y_real_var)
    amp_eval_var    = (np.exp(_y_real_var)-1) * np.exp(2*_y_real_mean+_y_real_var)
        
    # Phz
    phz_eval_mean = _y_imag_mean
    phz_eval_var = _y_imag_var
    
    return amp_eval_mean, amp_eval_var, phz_eval_mean, phz_eval_var

### Restore Run

In [38]:
amp_eval_mean, amp_eval_var, phz_eval_mean, phz_eval_var = GPRecoverEIS(y_eval_mean, y_eval_var)

C:\Users\Baihm\AppData\Local\Temp\3\ipykernel_17600\2899854138.py:10: RuntimeWarning: overflow encountered in exp
  amp_eval_var    = (np.exp(_y_real_var)-1) * np.exp(2*_y_real_mean+_y_real_var)
C:\Users\Baihm\AppData\Local\Temp\3\ipykernel_17600\2899854138.py:10: RuntimeWarning: overflow encountered in multiply
  amp_eval_var    = (np.exp(_y_real_var)-1) * np.exp(2*_y_real_mean+_y_real_var)


### Plot Restored Data

In [39]:
if True:
    n_freq = np.shape(y_eval_mean)[1]//2
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('rainbow_r')
    axis0 = fig.add_subplot(2,3,1)
    axis1 = fig.add_subplot(2,3,2)
    axis2 = fig.add_subplot(2,3,3, projection = '3d')
    axis3 = fig.add_subplot(2,3,4)
    axis4 = fig.add_subplot(2,3,5)
    axis5 = fig.add_subplot(2,3,6, projection = '3d')


    for i in range(np.shape(x_eval)[0]):
        axis1.plot(chData[0,0,:], amp_eval_mean[i,:], color = cmap(i/np.shape(x_eval)[0]))
        axis4.plot(chData[0,0,:], phz_eval_mean[i,:] * 180 / np.pi, color = cmap(i/np.shape(x_eval)[0]))
        
        axis0.fill_between(chData[0,0,:], amp_eval_mean[i,:] - 2*np.sqrt(amp_eval_var[i,:]), amp_eval_mean[i,:] + 2*np.sqrt(amp_eval_var[i,:]), 
                        alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
        axis3.fill_between(chData[0,0,:], phz_eval_mean[i,:] * 180 / np.pi - 2*np.sqrt(phz_eval_var[i,:]) * 180 / np.pi, phz_eval_mean[i,:] * 180 / np.pi + 2*np.sqrt(phz_eval_var[i,:]) * 180 / np.pi, 
                        alpha=0.2, color = cmap(i/np.shape(x_eval)[0]))
    
    # for i in range(np.shape(x_train)[0]):
    #     axis1.semilogx(chData[0,0,:], y_train[i,:n_freq], 'black', alpha = 0.3)
    #     axis4.semilogx(chData[0,0,:], y_train[i,n_freq:] * 180 / np.pi, 'black', alpha = 0.3)
    
    axis0.set_xscale('log')
    axis0.set_yscale('log')
    axis1.set_xscale('log')
    axis1.set_yscale('log')

    axis3.set_xscale('log')
    axis4.set_xscale('log')

    axis1.sharex(axis0)
    axis1.sharey(axis0)
    axis4.sharex(axis3)
    axis4.sharey(axis3)

    init_elev = 21  # 仰角
    init_azim = 55  # 方位角
    axis2.view_init(elev=init_elev, azim=init_azim)
    axis5.view_init(elev=init_elev, azim=init_azim)

    _x = np.arange(np.shape(x_eval)[0])
    _y = np.log10(chData[0,0,:]).flatten()
    X, Y = np.meshgrid(_x, _y, indexing='ij')
    axis2.plot_surface(X, Y, y_eval_mean[:,:n_freq], cmap='viridis_r', alpha=0.8)
    axis5.plot_surface(X, Y, y_eval_mean[:,n_freq:] * 180 / np.pi, cmap='viridis_r', alpha=0.8)




# Spline

In [276]:
def SplineataLoader(chData, eis_seq, SPEED_RATE = 2, LOG_FLAG = True):
    # Speed Rate = 10 means 1 day = 10 points
    x_day = [datetime.strptime(date, '%Y%m%d') for date in EISDict.keys()]
    x_day = [x_day[i] for i in eis_seq]

    x_train = np.array([(poi - x_day[0]).days for poi in x_day])
    x_eval = np.linspace(0,max(x_train),max(x_train)*SPEED_RATE+1)

    if LOG_FLAG:
        chData_log = np.log(chData[eis_seq,1,:] + 1j*chData[eis_seq,2,:])
        y_train = np.hstack([chData_log.real,chData_log.imag])
    else: 
        y_train = np.hstack([chData[eis_seq,1,:],chData[eis_seq,2,:]])

    logger.info(f"\nx: {np.shape(x_train)} \ny: {np.shape(y_train)} \nx_pred{np.shape(x_eval)}")

    return x_train, y_train, x_eval


## Spline Result Plot

In [277]:
if False:
    # for i in range(np.shape(y_train)[1]):
    Splie_interp = interp.CubicSpline(x_train, y_train)
    y_eval = Splie_interp(x_eval)
    n_freq = np.shape(y_eval)[0]//2
    fig = plt.figure(figsize=(8,8))
    cmap = plt.colormaps.get_cmap('viridis_r')
    axis0 = fig.add_subplot(1,3,1)
    axis1 = fig.add_subplot(1,3,2)
    axis2 = fig.add_subplot(1,3,3, projection='3d')
    for i in range(n_freq):
    # for i in [50]:
        
        axis0.plot(x_eval, y_eval[:,i], color = cmap(i/n_freq))
        axis0.plot(x_train, y_train[:,i], 'r.')
        
        j = i + n_freq
        axis1.plot(x_eval, y_eval[:,j], color = cmap(i/n_freq))
        axis1.plot(x_train, y_train[:,j], 'r.')

    axis0.set_xlabel('x')
    axis0.set_ylabel('y')
    axis0.set_title("Single-Point GPR: Mean and Variance")
    axis0.legend()



# Hyper Parameter Draft

## Rank

In [50]:
if True:
    from sklearn.decomposition import PCA
    from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding, MDS
    import umap.umap_ as umap  # 请确保安装了 umap-learn


    data = y_train.T
    # data = y_train[:,:n_freq].T
    # data = y_train.T

    _order = 3
    methods = {
        'PCA': PCA(n_components=_order),
        't-SNE': TSNE(n_components=_order, perplexity=5, random_state=42),  # perplexity 设置为 5
        'Isomap': Isomap(n_components=_order),
        'LLE': LocallyLinearEmbedding(n_components=_order, random_state=42),
        'MDS': MDS(n_components=_order, random_state=42),
        'UMAP': umap.UMAP(n_components=_order, random_state=42)
    }

    embeddings = {}
    emb_dist = {}
    for name, method in methods.items():
        embedding = method.fit_transform(data)
        embeddings[name] = embedding
        
        _x = embedding[:,0].flatten()
        _y = embedding[:,1].flatten()

        emb_dist[name] = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                            (_y[:, np.newaxis] - _y[np.newaxis, :])**2)
        


    fig, axis = plt.subplots(3,4,figsize=(12,6))
    for i, (name, emb) in enumerate(embeddings.items()):
        _x = emb[:,0].flatten()
        _y = emb[:,1].flatten()

        _dist = np.sqrt((_x[:, np.newaxis] - _x[np.newaxis, :])**2 + 
                            (_y[:, np.newaxis] - _y[np.newaxis, :])**2)
        # _dist = emb_dist[name]

        axis[np.int16(i/2),(i%2)*2].scatter(emb[:, 0], emb[:, 1], c=np.arange(np.shape(data)[0]), cmap='rainbow_r', edgecolor='k', s=100)
        axis[np.int16(i/2),(i%2)*2].set_title(name)

        s = axis[np.int16(i/2),(i%2)*2+1].imshow(_dist, cmap='coolwarm', interpolation='nearest')
        fig.colorbar(s, ax=axis[np.int16(i/2),(i%2)*2+1])
    plt.tight_layout()
    plt.show()


c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\sklearn\manifold\_isomap.py:384: UserWarning: The number of connected components of the neighbors graph is 2 > 1. Completing the graph to fit Isomap might be slow. Increase the number of neighbors to avoid this issue.
  self._fit_transform(X)
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\scipy\sparse\_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Baihm\anaconda3\envs\EISNN\Lib\site-packages\umap\umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Us